In [ ]:
!python3 -m pip install opencv-python ultralytics ipywidgets

In [ ]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is available
print(torch.cuda.device_count())  # Should print the number of GPUs available
print(torch.cuda.current_device())  # Should print the current device index (should be 0 if a GPU is used)
print(torch.cuda.get_device_name(0))  # Should print the name of the GPU (Jetson Nano has a Maxwell GPU)

In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available")
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available")

In [ ]:
import cv2
import torch
from ultralytics import YOLO
from IPython.display import display
import ipywidgets as widgets
import threading
import time

# Load the YOLOv10s model
model = YOLO('yolov10s.pt')
model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Define the GStreamer pipeline for the CSI camera
pipeline = (
    "nvarguscamerasrc ! "
    "video/x-raw(memory:NVMM), width=1280, height=720, framerate=30/1 ! "
    "nvvidconv flip-method=2 ! "
    "video/x-raw, format=BGRx ! "
    "videoconvert ! "
    "video/x-raw, format=BGR ! "
    "appsink"
)

# Initialize the video capture with the GStreamer pipeline
cap = cv2.VideoCapture(pipeline, cv2.CAP_GSTREAMER)

# Create an output widget
output_widget = widgets.Image()
display(output_widget)

def process_frame(frame):
    results = model(frame)
    annotated_frame = results[0].plot()
    return cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)

def video_feed():
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to read frame from camera. Exiting...")
            break
        processed_frame = process_frame(frame)
        _, jpeg = cv2.imencode('.jpg', processed_frame)
        output_widget.value = jpeg.tobytes()
        time.sleep(0.033)  # Approximately 30 FPS

# Start the video feed in a separate thread
thread = threading.Thread(target=video_feed)
thread.daemon = True
thread.start()
